In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('c:/fomo_data/model_data_1.parquet')

In [3]:
df.head()

,locationid,ridership,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2.0,1.0,2022-02-06 15:00:00,-5.6,0.0,0.0,11.0,0.0,0.0,35.0,6.600000,331.0
1,2.0,1.0,2022-02-16 23:00:00,5.4,0.0,0.0,29.0,0.0,0.0,97.0,12.100000,280.0
2,2.0,1.0,2022-02-23 08:00:00,13.5,0.0,0.0,44.0,8.0,11.0,100.0,16.299999,227.0
3,2.0,4.0,2022-02-23 19:00:00,17.5,0.0,0.0,22.0,5.0,2.0,54.0,21.600000,184.0
4,2.0,1.0,2022-03-22 10:00:00,7.4,0.0,0.0,20.0,0.0,30.0,8.0,10.800000,127.0


In [5]:
comments = pd.read_csv('c:/fomo_data/sentiment_scores1.csv')

In [8]:
comments.head()

,id,name,address,comment,created_at,latitude,longitude,sentiment
0,1,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281",hot yoga point . clear view across hudson weig...,2019-12-31 17:12:40,40.711974,-74.016237,1
1,2,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281","gym floor bit uninspiring , location great .",2019-12-17 14:00:01,40.711974,-74.016237,1
2,3,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281",location low ceiling get crowded morning,2019-01-14 04:06:52,40.711974,-74.016237,-1
3,4,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281",excellent ! class motivate well great equipment,2018-09-25 10:38:14,40.711974,-74.016237,1
4,5,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281","great view nice lounge area , overall pretty s...",2018-07-10 22:41:19,40.711974,-74.016237,1


In [9]:
comments = comments.drop(columns=['name', 'address', 'comment'])
comments.head()

,id,created_at,latitude,longitude,sentiment
0,1,2019-12-31 17:12:40,40.711974,-74.016237,1
1,2,2019-12-17 14:00:01,40.711974,-74.016237,1
2,3,2019-01-14 04:06:52,40.711974,-74.016237,-1
3,4,2018-09-25 10:38:14,40.711974,-74.016237,1
4,5,2018-07-10 22:41:19,40.711974,-74.016237,1


In [10]:
comments = comments.drop(columns=['id'])

In [11]:
import geopandas as gpd

#sahpe file for the taxi zones
taxi_zones = gpd.read_file('c:/fomo_data/taxi.shp')

# Convert subway latitude and longitude to a GeoDataFrame
geometry = gpd.points_from_xy(comments['longitude'], comments['latitude'])
comments_geo = gpd.GeoDataFrame(comments, geometry=geometry, crs="EPSG:4326")

# Reproject taxi_zones to the same CRS as subway_geo if needed
taxi_zones = taxi_zones.to_crs(comments_geo.crs)

# Perform a spatial join to find the taxi zone for each subway station using 'predicate' instead of 'op'
subway_geo = gpd.sjoin(comments_geo, taxi_zones, how='left', predicate='intersects')

In [12]:
subway_geo.head()

,created_at,latitude,longitude,sentiment,geometry,index_right,borough,location_i,objectid,shape_area,shape_leng,zone
0,2019-12-31 17:12:40,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
1,2019-12-17 14:00:01,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
2,2019-01-14 04:06:52,40.711974,-74.016237,-1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
3,2018-09-25 10:38:14,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
4,2018-07-10 22:41:19,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City


In [13]:
comments['location_id'] = subway_geo['location_i']

In [14]:
comments.head()

,created_at,latitude,longitude,sentiment,location_id
0,2019-12-31 17:12:40,40.711974,-74.016237,1,13.0
1,2019-12-17 14:00:01,40.711974,-74.016237,1,13.0
2,2019-01-14 04:06:52,40.711974,-74.016237,-1,13.0
3,2018-09-25 10:38:14,40.711974,-74.016237,1,13.0
4,2018-07-10 22:41:19,40.711974,-74.016237,1,13.0


In [15]:
comments.to_parquet('c:/fomo_data/comments_cleaned.parquet')

In [16]:
comments.drop(columns=['latitude', 'longitude'])

,created_at,sentiment,location_id
0,2019-12-31 17:12:40,1,13.0
1,2019-12-17 14:00:01,1,13.0
2,2019-01-14 04:06:52,-1,13.0
3,2018-09-25 10:38:14,1,13.0
4,2018-07-10 22:41:19,1,13.0
...,...,...,...
18156,2018-04-06 15:50:32,1,79.0
18157,2024-02-15 00:58:10,0,79.0
18158,2018-09-21 19:46:45,0,79.0
18159,2017-09-07 00:06:36,1,79.0
